In [15]:
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import (StartMeshcat, MeshcatVisualizer,
                         LeafSystem, AddMultibodyPlantSceneGraph,
                         JointSliders,
                         DiagramBuilder)
from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddPackagePaths

In [4]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
class PrintJacobian(LeafSystem): 
    
    def __init__(self, plant, frame):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._frame = frame
        self.DeclareVectorInputPort('state', plant.num_multibody_states())
        self.DeclareForcedPublishEvent(self.Publish)
        
    def Publish(self, context):
        

In [16]:
def pick_and_place_jacobians_example(): 
    
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    AddPackagePaths(parser)
    parser.AddAllModelsFromFile(FindResource("models/iiwa_and_wsg.dmd.yaml"))
    plant.Finalize()
    
    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )
    
    G = plant.GetBodyByName("body").body_frame()
    print_jacobian = builder.AddSystem(PrintJacobian(plant, G))
    builder.Connect(plant.get_state_output_port(),
                    print_jacobian.get_input_port())

    # If you want to set the initial positions manually, use this:
    #plant.SetPositions(plant.GetMyContextFromRoot(context),
    #                   plant.GetModelInstanceByName("iiwa"),
    #                   [0, 0, 0, 0, 0, 0, 0])

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()

In [ ]:
meshcat.DeleteAddedControls()
pick_and_place_jacobians_example()

INFO:drake:Press the 'Stop JointSliders' button in Meshcat or press 'Escape' to continue.
